In [1]:
import pvdeg
from pvdeg import TEST_DATA_DIR
import pandas as pd
import os
import pickle

## Pysam

rundown on pysam...

https://nrel-pysam.readthedocs.io/en/main/inputs-from-sam.html

In [2]:
weather_pvgis, meta_pvgis = pvdeg.weather.get(database="PVGIS", id=(25.783388, -80.189029))

In [3]:
results = pvdeg.standards.pysam_hourly_trivial(
    weather_df=weather_pvgis,
    meta=meta_pvgis,
)

In [ ]:
def pysam_annual_energy(weather_df, meta):
    results = pvdeg.standards.pysam_hourly_trivial(
        weather_df=weather_pvgis,
        meta=meta_pvgis,
    )

    return results['annual_energy']

In [ ]:

GEO_META = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-meta.csv"), index_col=0)

with open(os.path.join(TEST_DATA_DIR, "summit-weather.pkl"), "rb") as f:
    GEO_WEATHER = pickle.load(f)  # unchunked

template = pvdeg.geospatial.output_template(
    
    ds_gids=GEO_WEATHER, 
    shapes= {
        'Annual Energy': ("gid", ),
    }
)

geo_res = pvdeg.geospatial.analysis(
    weather_ds=GEO_WEATHER,
    meta_df=GEO_META,
    func=pysam_annual_energy,
    template=template
)

In [ ]:
weather_db = 'PSM3'
weather_id = (25.783388, -80.189029)
weather_arg = {'api_key': 'DEMO_KEY',
               'email': 'user@mail.com',
               'names': 'tmy',
               'attributes': [],
               'map_variables': True}

weather_df, meta = pvdeg.weather.get(weather_db, weather_id, **weather_arg)